## Jeopardy Fun Fact Analysis 



In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import tweepy
import json
import csv
import pandas as pd
import re

What do the fun facts say about jeopardy contestents?

Can we make any predictions / loose correlations (contestant type -- fun fact)

Do fun facts get worse/ better as you win more?

Likes on twitter 

#https://www.npr.org/2011/06/26/137429737/what-story-would-you-tell-on-jeopardy

## Jeopardy Archive Scrape 

In [2]:
#http://www.j-archive.com/showgame.php?game_id=6388

## Scrape online archive data
#Prep variables 
index = 0
output = []
archive_link = "http://www.j-archive.com/showgame.php?game_id="
game_id = 6389
jeopardy_archive_link = archive_link + str(game_id)


In [3]:
#start extraction 
while index < 500:
    
    # pull page
    page_response = requests.get(jeopardy_archive_link, timeout=5)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    
    # empty variables
    anecdotes = []
    final_scores = []
    names = []
    show_info1 = []
    show_info2 = []
    show_info3 = []
    
    #title date 
    title_date = page_content.find_all('title')[0].text # clean to just date (?)
    
    for j in range(0, 3):
        #Find all anecdotes for contestants 
        paragraphs = page_content.find_all("p")[j].text
        
        # Final all final scores for contestants 
        try:
            table1 = page_content.find_all(lambda tag: tag.name == 'td' and 
                                   tag.get('class') == ['score_positive'])[9:12][j].text
            pass
        except IndexError:
            print("error" + str(new_game_id))
        
        #find all names
        table2 = page_content.find_all(lambda tag: tag.name == 'td' and 
                                   tag.get('class') == ['score_player_nickname'])[j].text
        # append those players together
        anecdotes.append(paragraphs)
        final_scores.append(table1)
        names.append(table2)

    # reorder and correct data 
    show_info1.extend([names[0],anecdotes[2],final_scores[0],title_date])
    show_info2.extend([names[1],anecdotes[1],final_scores[1],title_date])
    show_info3.extend([names[2],anecdotes[0],final_scores[2],title_date])
    
    #create output file
    output.append(show_info1)
    output.append(show_info2)
    output.append(show_info3)
    
    #create link to next page
        #create previous page number
    new_game_id = page_content.find_all(lambda tag: tag.name == 'a' and 
                                        tag.get('href') and 
                                        tag.text == "[<< previous game]")
    new_game_id = re.findall(r'\d+', str(new_game_id[0]))[0]
        # create link 
    jeopardy_archive_link = archive_link + new_game_id
    jeopardy_archive_link
    
    #update iterator 
    index = index + 1

#output

error6309
error6309
error6291
error6291
error6227
error6227
error6227
error6226
error6226
error6226
error6224
error6224
error6224
error6223
error6223
error6223
error6176
error6163
error6141
error6141
error6041
error6004
error5913
error5913
error5786


In [134]:
arch = pd.DataFrame.from_records(output)
arch.columns = ["Nickname", "Player Details", "Final Score", "Show Info"]
arch

1500

In [147]:
arch["Player Details"].iloc[0].split("from")[1].split("(")[1].strip()

'whose 5-day cash winnings total $109,700)'

In [148]:
# Create Name Field 

# SPlIT TWT TEXT INTO SEPERATE COLUMNS 
arch["Full Name"] = ""
arch["Occupation"] = ""
arch["Hometown"] = ""
arch["Streak"] = ""
arch["Cash Winnings"] = ""
arch["Archive Info"] = ""
arch["Date"] = ""

for i in range(0,len(arch)):
    
    # extract player details 
    arch.at[i,"Full Name"] = arch["Player Details"].iloc[i].split(",")[0]
    arch.at[i,"Occupation"] = arch["Player Details"].iloc[i].split(",")[1].split("from")[0].strip()
    arch.at[i,"Hometown"] = arch["Player Details"].iloc[i].split("from")[1].split("(")[0].strip()
    
    # To Add 
    #arch.at[i,"Streak"] = ""
    #arch.at[i,"Cash Winnings"] = ""
    
    # extra show info 
    arch.at[i,"Archive Info"] = arch["Show Info"].iloc[i].split(", aired")[0].strip()
    arch.at[i,"Date"] = arch["Show Info"].iloc[i].split(", aired")[1].strip()

arch.head(5)


,Nickname,Player Details,Final Score,Show Info,Full Name,Occupation,Hometown,Archive Info,Date,Streak,Cash Winnings
0,Jason,"Jason Zuffranieri, a math teacher from Albuque...","$27,600","J! Archive - Show #8045, aired 2019-07-26",Jason Zuffranieri,a math teacher,"Albuquerque, New Mexico",J! Archive - Show #8045,2019-07-26,,
1,Maggie,"Maggie Lehrman, an editor and writer from Broo...",$0,"J! Archive - Show #8045, aired 2019-07-26",Maggie Lehrman,an editor and writer,"Brooklyn, New York",J! Archive - Show #8045,2019-07-26,,
2,Michael,"Michael Riggs, an educational therapist from T...",$2,"J! Archive - Show #8045, aired 2019-07-26",Michael Riggs,an educational therapist,"Tustin, California",J! Archive - Show #8045,2019-07-26,,
3,Jason,"Jason Zuffranieri, a math teacher from Albuque...","$4,400","J! Archive - Show #8044, aired 2019-07-25",Jason Zuffranieri,a math teacher,"Albuquerque, New Mexico",J! Archive - Show #8044,2019-07-25,,
4,Eric,"Eric Kaplan, a retired OB-GYN physician from L...",$0,"J! Archive - Show #8044, aired 2019-07-25",Eric Kaplan,a retired OB-GYN physician,"Long Beach, California",J! Archive - Show #8044,2019-07-25,,


In [149]:
# - Clean date
arch['Date'].replace(regex=True,inplace=True,to_replace=r':',value=r'')
arch['Date'] = pd.to_datetime(arch['Date'], errors='coerce')
arch.head(5)

,Nickname,Player Details,Final Score,Show Info,Full Name,Occupation,Hometown,Archive Info,Date,Streak,Cash Winnings
0,Jason,"Jason Zuffranieri, a math teacher from Albuque...","$27,600","J! Archive - Show #8045, aired 2019-07-26",Jason Zuffranieri,a math teacher,"Albuquerque, New Mexico",J! Archive - Show #8045,2019-07-26,,
1,Maggie,"Maggie Lehrman, an editor and writer from Broo...",$0,"J! Archive - Show #8045, aired 2019-07-26",Maggie Lehrman,an editor and writer,"Brooklyn, New York",J! Archive - Show #8045,2019-07-26,,
2,Michael,"Michael Riggs, an educational therapist from T...",$2,"J! Archive - Show #8045, aired 2019-07-26",Michael Riggs,an educational therapist,"Tustin, California",J! Archive - Show #8045,2019-07-26,,
3,Jason,"Jason Zuffranieri, a math teacher from Albuque...","$4,400","J! Archive - Show #8044, aired 2019-07-25",Jason Zuffranieri,a math teacher,"Albuquerque, New Mexico",J! Archive - Show #8044,2019-07-25,,
4,Eric,"Eric Kaplan, a retired OB-GYN physician from L...",$0,"J! Archive - Show #8044, aired 2019-07-25",Eric Kaplan,a retired OB-GYN physician,"Long Beach, California",J! Archive - Show #8044,2019-07-25,,


In [5]:
arch.to_csv('jeopardy_archive_data.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


## twitter webscrape

In [6]:
jeopardy_funfact_twitter_link = 'https://twitter.com/cooljepstories?lang=en'

In [7]:
# load Twitter API credentials
consumer_key= 'arBjlD4kgWVTuqk4pIXw0JWPH'
consumer_secret= 'gH16KBtOAjrEZ5Mn0TzfvLaODG1yMkDOXGlwlGOl2ptrLX43kN'
access_key = '1119604302-tnGJbpmtWLbVZ6WltX6WBlnK7rC7kBTWD7NiM7g'
access_secret= 'XMwoahiPfA0bGnX9xitN5AJzn9UzLiq8hZCoCtx7TaI5K'

#User ID
userID = "@CoolJepStories"

# Authorization to consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
# Access to user's access key and access secret 
auth.set_access_token(access_key, access_secret) 
# Calling api 
api = tweepy.API(auth) 

# 1750 tweets to be extracted 
number_of_tweets=1750
tweets = api.user_timeline(screen_name=userID, count = number_of_tweets, 
                           include_rts = False, tweet_mode="extended") 



In [8]:
for info in tweets[:1]:
     print("ID: {}".format(info.id))
     print(info.created_at)
     print(info.full_text)
     print("\n")

ID: 1194098342285991938
2019-11-12 03:43:24
The coolest Jeopardy! story.
#WeLoveYouAlex https://t.co/P02SrYHoO0




In [9]:
all_tweets = []
all_tweets.extend(tweets)
oldest_id = tweets[-1].id
while True:
    tweets = api.user_timeline(screen_name=userID, 
                           count=200, # max allowed count
                           include_rts = False,
                           max_id = oldest_id - 1,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    if len(tweets) == 0:
        break
    oldest_id = tweets[-1].id
    all_tweets.extend(tweets)
    print('N of tweets downloaded till now {}'.format(len(all_tweets)))

N of tweets downloaded till now 392
N of tweets downloaded till now 589
N of tweets downloaded till now 788
N of tweets downloaded till now 986
N of tweets downloaded till now 1170
N of tweets downloaded till now 1327
N of tweets downloaded till now 1490
N of tweets downloaded till now 1620


In [94]:
#transform the tweepy tweets into a 2D array that will populate the csv	
from pandas import DataFrame
outtweets = [[tweet.id_str, 
              tweet.created_at, 
              tweet.favorite_count, 
              tweet.retweet_count, 
              tweet.full_text.encode("utf-8").decode("utf-8")] 
             for idx,tweet in enumerate(all_tweets)]
df = DataFrame(outtweets,columns=["id","created_at","favorite_count","retweet_count", "text"])
df.to_csv('%s_tweets.csv' % userID,index=False)
df.head(10)

,id,created_at,favorite_count,retweet_count,text
0,1194098342285991938,2019-11-12 03:43:24,78,13,The coolest Jeopardy! story.\n#WeLoveYouAlex h...
1,1154934362737205253,2019-07-27 01:59:44,23,2,"7/26/19:\n""My medical school interview was wit..."
2,1154749896022548481,2019-07-26 13:46:44,12,1,If you are interested in tipping a hat of grat...
3,1154749838606688257,2019-07-26 13:46:30,165,8,Today is the final day @CoolJepStories posts a...
4,1154567652465106944,2019-07-26 01:42:33,6,1,"7/25/19:\n""I like cooking with my 2-year-old s..."
5,1154498659637051393,2019-07-25 21:08:24,8,3,7/24/19:\n“I played drums in a rhythmic troupe...
6,1154153044772306945,2019-07-24 22:15:03,5,0,"7/23/19:\n""I like telling dad jokes.""\n""I run ..."
7,1154141731669073920,2019-07-24 21:30:06,6,0,"7/22/19:\n""I explain forensic science to kids ..."
8,1154129140758536192,2019-07-24 20:40:04,5,0,"7/19/19:\n""I deliberately struck out so we cou..."
9,1154117827709886464,2019-07-24 19:55:07,8,2,"7/18/19:\n""I dyed my hair blue &amp; white for..."


## Clean text 


In [95]:
# Flag just relevant tweets 
df["game_info_flg"] = ""

for i in range(0,len(df)):
    #print(((df['text'][i][0]).isnumeric()))
    df.at[i,'game_info_flg'] = df['text'][i][0].isnumeric()

df.head(5)
len(df)

1620

In [97]:
df_games = df[df["game_info_flg"] == True]
df_games.reset_index(drop=True, inplace=True)
df_games.head(5)

,id,created_at,favorite_count,retweet_count,text,game_info_flg
0,1154934362737205253,2019-07-27 01:59:44,23,2,"7/26/19:\n""My medical school interview was wit...",True
1,1154567652465106944,2019-07-26 01:42:33,6,1,"7/25/19:\n""I like cooking with my 2-year-old s...",True
2,1154498659637051393,2019-07-25 21:08:24,8,3,7/24/19:\n“I played drums in a rhythmic troupe...,True
3,1154153044772306945,2019-07-24 22:15:03,5,0,"7/23/19:\n""I like telling dad jokes.""\n""I run ...",True
4,1154141731669073920,2019-07-24 21:30:06,6,0,"7/22/19:\n""I explain forensic science to kids ...",True


In [98]:
len(df_games)

1409

In [101]:
# SPlIT TWT TEXT INTO SEPERATE COLUMNS 
df_games["Date"] = ""
df_games["Answer1"] = ""
df_games["Answer2"] = ""
df_games["Answer3"] = ""

for i in range(0,len(df_games)):
    
    twt_txt = df_games["text"].iloc[i].split("\n")
    # split and save as a new row 
    df_games.at[i,"Date"] = twt_txt[0]
    df_games.at[i,"Answer1"] = twt_txt[1]
    df_games.at[i,"Answer2"] = twt_txt[2]
    if(len(twt_txt) > 3):
        df_games.at[i,"Answer3"] = twt_txt[3]
df_games.head(5)

C:\Users\jwilson2\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jwilson2\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jwilson2\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on 

,id,created_at,favorite_count,retweet_count,text,game_info_flg,Date,Answer1,Answer2,Answer3
0,1154934362737205253,2019-07-27 01:59:44,23,2,"7/26/19:\n""My medical school interview was wit...",True,7/26/19:,"""My medical school interview was with a @Jeopa...","""The children's' book industry is phenomenal.""","""A movie editor in France has a similar name t..."
1,1154567652465106944,2019-07-26 01:42:33,6,1,"7/25/19:\n""I like cooking with my 2-year-old s...",True,7/25/19:,"""I like cooking with my 2-year-old son.""","""Chuck Norris' aunt thought I looked justl ike...","""It took me seconds to 'win' a game of anti-ch..."
2,1154498659637051393,2019-07-25 21:08:24,8,3,7/24/19:\n“I played drums in a rhythmic troupe...,True,7/24/19:,“I played drums in a rhythmic troupe.”,“I had to memorize the Gettysburg Address.”,"“In Mexico, I was mistaken for Nicolas Cage.”"
3,1154153044772306945,2019-07-24 22:15:03,5,0,"7/23/19:\n""I like telling dad jokes.""\n""I run ...",True,7/23/19:,"""I like telling dad jokes.""","""I run an online forum to talk about the neigh...","""My stuffed manatee is a comfort animal in my ..."
4,1154141731669073920,2019-07-24 21:30:06,6,0,"7/22/19:\n""I explain forensic science to kids ...",True,7/22/19:,"""I explain forensic science to kids using the ...","""I went to Spain &amp; ran into a lady who was...","""I travel around the world playing Sudoku."""


In [102]:
df_games.dtypes

id                        object
created_at        datetime64[ns]
favorite_count             int64
retweet_count              int64
text                      object
game_info_flg             object
Date                      object
Answer1                   object
Answer2                   object
Answer3                   object
dtype: object

In [103]:
# CLEAN COLUMNS 
# - Clean date
df_games['Date'].replace(regex=True,inplace=True,to_replace=r':',value=r'')
df_games['Date'] = pd.to_datetime(df_games['Date'], errors='coerce')

df_games.head(5)

C:\Users\jwilson2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\jwilson2\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,id,created_at,favorite_count,retweet_count,text,game_info_flg,Date,Answer1,Answer2,Answer3
0,1154934362737205253,2019-07-27 01:59:44,23,2,"7/26/19:\n""My medical school interview was wit...",True,2019-07-26,"""My medical school interview was with a @Jeopa...","""The children's' book industry is phenomenal.""","""A movie editor in France has a similar name t..."
1,1154567652465106944,2019-07-26 01:42:33,6,1,"7/25/19:\n""I like cooking with my 2-year-old s...",True,2019-07-25,"""I like cooking with my 2-year-old son.""","""Chuck Norris' aunt thought I looked justl ike...","""It took me seconds to 'win' a game of anti-ch..."
2,1154498659637051393,2019-07-25 21:08:24,8,3,7/24/19:\n“I played drums in a rhythmic troupe...,True,2019-07-24,“I played drums in a rhythmic troupe.”,“I had to memorize the Gettysburg Address.”,"“In Mexico, I was mistaken for Nicolas Cage.”"
3,1154153044772306945,2019-07-24 22:15:03,5,0,"7/23/19:\n""I like telling dad jokes.""\n""I run ...",True,2019-07-23,"""I like telling dad jokes.""","""I run an online forum to talk about the neigh...","""My stuffed manatee is a comfort animal in my ..."
4,1154141731669073920,2019-07-24 21:30:06,6,0,"7/22/19:\n""I explain forensic science to kids ...",True,2019-07-22,"""I explain forensic science to kids using the ...","""I went to Spain &amp; ran into a lady who was...","""I travel around the world playing Sudoku."""


# Join Tables

In [128]:
# JOIN TABLES 
result = pd.merge(df_games,arch, on='Date')
result.head(10)




,id,created_at,favorite_count,retweet_count,text,game_info_flg,Date,Answer1,Answer2,Answer3,Nickname,Player Details,Final Score,Show Info,Full Name,Occupation,Hometown,Archive Info
0,871865985694871552,2017-06-05 23:07:04,21,1,"6/5/17:\n""I can read hieroglyphics.""\n""I drug ...",True,2017-06-05,"""I can read hieroglyphics.""","""I drug test truck drivers and railroad workers.""","""I lived with four women in law school.""",Jason,"Jason Zuffranieri, a math teacher from Albuque...","$27,600","J! Archive - Show #8045, aired 2019-07-26",Chantelle Schofield,an E.R. social worker,"Fort Myers, Florida",J! Archive - Show #7546
1,871865985694871552,2017-06-05 23:07:04,21,1,"6/5/17:\n""I can read hieroglyphics.""\n""I drug ...",True,2017-06-05,"""I can read hieroglyphics.""","""I drug test truck drivers and railroad workers.""","""I lived with four women in law school.""",Maggie,"Maggie Lehrman, an editor and writer from Broo...",$0,"J! Archive - Show #8045, aired 2019-07-26",Chantelle Schofield,an E.R. social worker,"Fort Myers, Florida",J! Archive - Show #7546
2,871865985694871552,2017-06-05 23:07:04,21,1,"6/5/17:\n""I can read hieroglyphics.""\n""I drug ...",True,2017-06-05,"""I can read hieroglyphics.""","""I drug test truck drivers and railroad workers.""","""I lived with four women in law school.""",Michael,"Michael Riggs, an educational therapist from T...",$2,"J! Archive - Show #8045, aired 2019-07-26",Chantelle Schofield,an E.R. social worker,"Fort Myers, Florida",J! Archive - Show #7546
3,871865985694871552,2017-06-05 23:07:04,21,1,"6/5/17:\n""I can read hieroglyphics.""\n""I drug ...",True,2017-06-05,"""I can read hieroglyphics.""","""I drug test truck drivers and railroad workers.""","""I lived with four women in law school.""",Jason,"Jason Zuffranieri, a math teacher from Albuque...","$4,400","J! Archive - Show #8044, aired 2019-07-25",Chantelle Schofield,an E.R. social worker,"Fort Myers, Florida",J! Archive - Show #7546
4,871865985694871552,2017-06-05 23:07:04,21,1,"6/5/17:\n""I can read hieroglyphics.""\n""I drug ...",True,2017-06-05,"""I can read hieroglyphics.""","""I drug test truck drivers and railroad workers.""","""I lived with four women in law school.""",Eric,"Eric Kaplan, a retired OB-GYN physician from L...",$0,"J! Archive - Show #8044, aired 2019-07-25",Chantelle Schofield,an E.R. social worker,"Fort Myers, Florida",J! Archive - Show #7546
5,871865985694871552,2017-06-05 23:07:04,21,1,"6/5/17:\n""I can read hieroglyphics.""\n""I drug ...",True,2017-06-05,"""I can read hieroglyphics.""","""I drug test truck drivers and railroad workers.""","""I lived with four women in law school.""",Shari,"Shari Meyer, a high school English teacher fro...",$0,"J! Archive - Show #8044, aired 2019-07-25",Chantelle Schofield,an E.R. social worker,"Fort Myers, Florida",J! Archive - Show #7546
6,871865985694871552,2017-06-05 23:07:04,21,1,"6/5/17:\n""I can read hieroglyphics.""\n""I drug ...",True,2017-06-05,"""I can read hieroglyphics.""","""I drug test truck drivers and railroad workers.""","""I lived with four women in law school.""",Jason,"Jason Zuffranieri, a math teacher from Albuque...","$30,000","J! Archive - Show #8043, aired 2019-07-24",Chantelle Schofield,an E.R. social worker,"Fort Myers, Florida",J! Archive - Show #7546
7,871865985694871552,2017-06-05 23:07:04,21,1,"6/5/17:\n""I can read hieroglyphics.""\n""I drug ...",True,2017-06-05,"""I can read hieroglyphics.""","""I drug test truck drivers and railroad workers.""","""I lived with four women in law school.""",Michelle,"Michelle Bruck, an attorney from Levittown, Pe...","$3,000","J! Archive - Show #8043, aired 2019-07-24",Chantelle Schofield,an E.R. social worker,"Fort Myers, Florida",J! Archive - Show #7546
8,871865985694871552,2017-06-05 23:07:04,21,1,"6/5/17:\n""I can read hieroglyphics.""\n""I drug ...",True,2017-06-05,"""I can read hieroglyphics.""","""I drug test truck drivers and railroad workers.""","""I lived with four women in law school.""",Corin,"Corin

In [16]:
# TEXT ANALYSIS OF ANECDOTES 
import nltk
import os
import nltk.corpus




In [18]:
# SUMMARY STATISTICS 





In [19]:
an.to_csv('jeopardy_twitter_data.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


NameError: name 'an' is not defined